## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
'''
未使用GridSearchCV
'''

# 讀取鳶尾花資料集
iris = datasets.load_iris()
x_train, x_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.25, random_state=6)

# 建立模型
GBR = GradientBoostingRegressor()

# 計算模型誤差 MSE
GBR.fit(x_train, y_train)
y_pred = GBR.predict(x_test)
MSE = round(mean_squared_error(y_test, y_pred), 5)
print('MSE: ', MSE)

MSE:  0.07117


In [3]:
'''
使用GridSearchCV
'''
# 設定要訓練的超參數組合
n_estimators = [5, 10, 20, 100, 200, 300]
max_depth = [1, 3, 5, 7, 9, 11, 13]
min_samples_leaf= [3, 6, 9, 12, 15, 18, 21]
max_depth = [1, 3, 5, 7, 9]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth, min_samples_leaf=min_samples_leaf)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(GBR, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=5)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 5 folds for each of 210 candidates, totalling 1050 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 672 tasks      | elapsed:    5.6s


Best Accuracy: -0.031798 using {'max_depth': 3, 'min_samples_leaf': 9, 'n_estimators': 100}


[Parallel(n_jobs=-1)]: Done 1050 out of 1050 | elapsed:    9.7s finished


In [4]:
# 使用最佳參數重新建立模型
GBR_bestparam = GradientBoostingRegressor(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'],
                                         min_samples_leaf=grid_result.best_params_['min_samples_leaf'])

# 訓練模型
GBR_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = GBR_bestparam.predict(x_test)

# 計算模型誤差 MSE
GBR.fit(x_train, y_train)
y_pred = GBR.predict(x_test)
MSE = round(mean_squared_error(y_test, y_pred), 5)
print('MSE: ', MSE)

MSE:  0.07529
